Ensure no data leaking by replacing post game stats with previous averaging

In [8]:
import pandas as pd

path = "../raw/schedules_raw.csv"

df = pd.read_csv(path)
df.head()

,game_id,season,week,date,home_team,away_team,home_score,away_score,Winner,home_pass_cmp,...,surface,temp,wind,game_type,weekday,gametime,location,home_coach,away_coach,referee
0,2014_01_GB_SEA,2014,1,9/4/2014,SEA,GB,36,16,SEA,19.0,...,fieldturf,71.0,11.0,REG,Thursday,20:30,Home,Pete Carroll,Mike McCarthy,John Parry
1,2014_01_NO_ATL,2014,1,9/7/2014,ATL,NO,37,34,ATL,31.0,...,fieldturf,NaN,NaN,REG,Sunday,13:00,Home,Mike Smith,Sean Payton,Bill Leavy
2,2014_01_CIN_BAL,2014,1,9/7/2014,BAL,CIN,16,23,CIN,35.0,...,sportturf,74.0,8.0,REG,Sunday,13:00,Home,John Harbaugh,Marvin Lewis,Gene Stetatore
3,2014_01_BUF_CHI,2014,1,9/7/2014,CHI,BUF,20,23,BUF,34.0,...,grass,74.0,3.0,REG,Sunday,13:00,Home,Marc Trestman,Doug Marrone,Brad Allen
4,2014_01_WAS_HOU,2014,1,9/7/2014,HOU,WAS,17,6,HOU,14.0,...,grass,NaN,NaN,REG,Sunday,13:00,Home,Bill O'Brien,Jay Gruden,Jerome Boger


Calculate metrics using scheduling data
1. Have 2 rows per game, one representing each team
2. Replace game data with previous week's data - model will only have access to previous week's data

In [9]:
import pandas as pd

def to_team_games(df: pd.DataFrame) -> pd.DataFrame:
    # clean any stray whitespace in headers
    df = df.copy()
    df.columns = df.columns.str.strip()

    # columns we must NOT rename
    special_keep = {"home_team","away_team","home_score","away_score"}

    # detect prefixed columns
    home_cols_all = [c for c in df.columns if c.startswith("home_")]
    away_cols_all = [c for c in df.columns if c.startswith("away_")]

    # stats to rename (exclude team name/score)
    home_stats = [c for c in home_cols_all if c not in special_keep]
    away_stats = [c for c in away_cols_all if c not in special_keep]

    # everything else = base/meta (date, lines, stadium, roof, refs, etc.)
    base_cols = [c for c in df.columns if c not in (home_stats + away_stats)]

    # helpers
    def swap_prefix(cols, old, new):
        return {c: c.replace(old, new, 1) for c in cols}

    # HOME perspective
    home_side = (
        df[base_cols + home_stats + away_stats]
        .rename(columns={
            **swap_prefix(home_stats, "home_", "team_"),
            **swap_prefix(away_stats, "away_", "opp_"),
        })
        .assign(
            team=lambda d: d["home_team"],
            opponent=lambda d: d["away_team"],
            team_score=lambda d: d["home_score"],
            opp_score=lambda d: d["away_score"],
            is_home=1,
        )
    )

    # AWAY perspective
    away_side = (
        df[base_cols + home_stats + away_stats]
        .rename(columns={
            **swap_prefix(away_stats, "away_", "team_"),
            **swap_prefix(home_stats, "home_", "opp_"),
        })
        .assign(
            team=lambda d: d["away_team"],
            opponent=lambda d: d["home_team"],
            team_score=lambda d: d["away_score"],
            opp_score=lambda d: d["home_score"],
            is_home=0,
        )
    )

    team_games = pd.concat([home_side, away_side], ignore_index=True)

    # convenient targets
    team_games["team_win"]  = (team_games["team_score"] > team_games["opp_score"]).astype(int)
    team_games["point_diff"] = team_games["team_score"] - team_games["opp_score"]

    # order columns: core → meta → team_* → opp_*
    core = ["game_id","season","week","date","team","opponent","is_home",
            "team_score","opp_score","team_win","point_diff"]
    meta = [c for c in base_cols if c not in core]
    team_stats = sorted([c for c in team_games.columns if c.startswith("team_")])
    opp_stats  = sorted([c for c in team_games.columns if c.startswith("opp_")])

    ordered = [c for c in core if c in team_games.columns] + meta + team_stats + opp_stats
    return team_games[ordered].sort_values(["season","week","team"]).reset_index(drop=True)

# usage:
# df = pd.read_csv("../raw/schedules_raw.csv")
df = to_team_games(df)

Roll back features to ensure model only has access to past data, adding prior_stats columns as necessary

In [10]:
def add_prior_features(
    team_games: pd.DataFrame,
    cols=None,
    group_keys=("team","season"),
    order_keys=("season","week","date"),
    lags=(1,),
    name_style="auto",
    fill=None
):
    g = team_games.copy()

    # ensure proper sort for shifting
    g["week"] = pd.to_numeric(g["week"], errors="ignore")
    g = g.sort_values(list(group_keys) + list(order_keys))

    # default: all numeric team_* columns
    if cols is None:
        cols = [c for c in g.columns if c.startswith("team_") and pd.api.types.is_numeric_dtype(g[c])]

    def prior_name(col, lag):
        if name_style == "suffix":
            return f"{col}_prior{lag}"
        if col.startswith("team_"):
            return col.replace("team_", f"team_prior{'' if lag==1 else f'{lag}_'}", 1)
        return f"{col}_prior{lag}"

    # compute lags per team-season
    for lag in lags:
        lagged = (
            g.groupby(list(group_keys), group_keys=False)[cols]
             .shift(lag)
             .rename(columns={c: prior_name(c, lag) for c in cols})
        )
        g = pd.concat([g, lagged], axis=1)

    # optional fill for first game(s) of season
    if fill is not None:
        new_cols = [prior_name(c, lag) for c in cols for lag in lags]
        if fill == "ffill":
            g[new_cols] = (
                g.groupby(list(group_keys), group_keys=False)[new_cols]
                 .apply(lambda x: x.ffill())
            )
        else:
            g[new_cols] = g[new_cols].fillna(fill)

    # --- NEW: drop duplicate columns ---
    g = g.loc[:, ~g.columns.duplicated()]

    return g

# ---- Team stats to lag ----
team_cols_to_lag = [
    "team_pass_att",
    "team_pass_cmp",
    "team_pass_yds",
    "team_pass_td",
    "team_pass_int",
    "team_pass_sacked",
    "team_pass_sacked_yds",
    
    "team_rush_att",
    "team_rush_yds",
    "team_rush_td",
    
    "team_first_down",
    "team_turnovers",
    
    "team_penalties",
    "team_penalties_yds",
    
    "team_fga", "team_fgm",       # field goals
    "team_xpa", "team_xpm",       # extra points
    
    "team_punt",
    "team_punt_yds",
    
    "team_plays_offense",
    "team_score"
]

# ---- Opponent stats to lag ----
opp_cols_to_lag = [
    "opp_pass_att",
    "opp_pass_cmp",
    "opp_pass_yds",
    "opp_pass_td",
    "opp_pass_int",
    "opp_pass_sacked",
    "opp_pass_sacked_yds",
    
    "opp_rush_att",
    "opp_rush_yds",
    "opp_rush_td",
    
    "opp_first_down",
    "opp_turnovers",
    
    "opp_penalties",
    "opp_penalties_yds",
    
    "opp_fga", "opp_fgm",
    "opp_xpa", "opp_xpm",
    
    "opp_punt",
    "opp_punt_yds",
    
    "opp_plays_offense",
    "opp_score"
]

# Team priors
df = add_prior_features(
    df,
    cols=team_cols_to_lag,
    group_keys=("team","season"),
    name_style="suffix"
)

# Opponent priors
df = add_prior_features(
    df,
    cols=opp_cols_to_lag,
    group_keys=("opponent","season"),
    name_style="suffix"
)

/var/folders/75/4m90t2xd47x2jvpz7fgpyfkh0000gn/T/ipykernel_45788/1567635445.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  g["week"] = pd.to_numeric(g["week"], errors="ignore")
/var/folders/75/4m90t2xd47x2jvpz7fgpyfkh0000gn/T/ipykernel_45788/1567635445.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  g["week"] = pd.to_numeric(g["week"], errors="ignore")


# 📊 NFL Gamelog Feature Engineering & Imputation Plan

## 1. Imputation Needs

| Column(s) | Issue | Strategy |
|-----------|-------|----------|
| `temp`, `wind` | Missing for domes | Impute with `0` or add categorical flag `indoor` |
| `roof`, `surface` | Inconsistent strings | Standardize categories (e.g., `closed`, `dome`, `outdoors`) |
| `*_prior1` (e.g., `team_pass_att_prior1`, `opp_pass_yds_prior1`) | Missing for Week 1 / bye weeks | Fill with previous season average, league average, or leave as NaN (tree models handle) |
| `spread_line`, `total_line`, `moneyline` | Missing in some games | Impute with market averages or drop rows if rare |
| `fga`, `fgm`, `xpa`, `xpm` | Sometimes `0` | Confirm 0 = true attempt (not missing); leave as-is |
| `penalties`, `fumbles` | 0 may look like missing | Standardize to keep 0 as valid |
| `referee`, `team_coach`, `opp_coach` | Sometimes missing | Encode `"Unknown"` or drop if not predictive |

---

## 2. Engineered Metrics

### Team Strength (per team, rolling/seasonal)
- Avg points scored / allowed  
- Avg total yards, pass yards, rush yards  
- Avg turnovers committed / forced  
- Avg first downs  
- Avg penalties & penalty yards  
- Avg sacks allowed / sacks generated  

### Contextual Features
- Rest days since last game  
- Home/away split performance (win % home vs away)  
- Travel distance to opponent stadium  
- Weather category: cold, hot, windy, indoor  

### Momentum & Form
- Current win/loss streak length  
- Average point differential in last 3 games  
- Trend of point differential (improving/declining)  

### Betting Market
- Spread & total line movement (if open/close available)  
- Moneyline implied win probability  

### Matchup-Specific Metrics
- Rush yards per game (team) vs rush yards allowed (opponent)  
- Pass yards per game (team) vs pass yards allowed (opponent)  
- Turnover margin differential (team vs opponent)  

### Efficiency & Advanced Metrics
- Yards per play  
- Pass completion %  
- Red zone TD conversion %  
- 3rd down conversion rate  
- Composite ratings: ELO, SRS, or custom power index  

---

## 3. Special Handling for Week 1
- `prior1` stats will be missing:  
  - Fill with **previous season averages** for that team  
  - If no prior season (dataset starts at 2014), use **league average**  
- Optionally drop Week 1 rows from training if you want max purity  

---

✅ **Bottom line:**  
- Keep all raw stats but replace in-season “post-game” stats with lag features.  
- Impute thoughtfully for Week 1 & special cases.  
- Engineer rolling averages, matchup diffs, and efficiency metrics for predictive strength.  

In [11]:
#impute averages

#impute probablitiy feature without vig
import numpy as np

def moneyline_to_prob(ml):
    """Convert American odds to implied probability (with vig)."""
    if ml < 0:
        return -ml / (-ml + 100)
    else:
        return 100 / (ml + 100)

def remove_vig_prob(team_ml, opp_ml):
    """
    Convert team/opponent moneylines into normalized probabilities.
    Returns (team_prob, opp_prob) with vig removed.
    """
    p_team = moneyline_to_prob(team_ml)
    p_opp = moneyline_to_prob(opp_ml)
    total = p_team + p_opp
    return p_team / total, p_opp / total

# --- Apply to your DataFrame ---
df["team_prob_novig"], df["opp_prob_novig"] = zip(
    *df[["team_moneyline", "opp_moneyline"]].apply(
        lambda x: remove_vig_prob(x["team_moneyline"], x["opp_moneyline"]), axis=1
    )
)

General Cleaning
1. Check for low variability columns
2. Check for missingness

In [12]:
print("Constant columns:", df.columns[df.nunique(dropna=True) <= 1].tolist())

const_cols = [
    'team_pass_sacked_yds', 'team_punt', 'team_punt_yds',
    'opp_pass_sacked_yds', 'opp_punt', 'opp_punt_yds',
    'team_pass_sacked_yds_prior1', 'team_punt_prior1', 'team_punt_yds_prior1',
    'opp_pass_sacked_yds_prior1', 'opp_punt_prior1', 'opp_punt_yds_prior1'
]

df = df.drop(columns=const_cols)

print("Constant columns:", df.columns[df.nunique(dropna=True) <= 1].tolist())

df = df.sort_values(["season", "week", "team"]).reset_index(drop=True)

Constant columns: ['team_pass_sacked_yds', 'team_punt', 'team_punt_yds', 'opp_pass_sacked_yds', 'opp_punt', 'opp_punt_yds', 'team_pass_sacked_yds_prior1', 'team_punt_prior1', 'team_punt_yds_prior1', 'opp_pass_sacked_yds_prior1', 'opp_punt_prior1', 'opp_punt_yds_prior1']
Constant columns: []


Adding feature for number of rest days since last game: rest_days

In [13]:
import numpy as np

# Make sure date is datetime
df['date'] = pd.to_datetime(df['date'])

# Sort so diffs are correct
df = df.sort_values(['team', 'season', 'week', 'date'])

# Previous game date and season for each team
df['__prev_date'] = df.groupby('team')['date'].shift(1)
df['__prev_season'] = df.groupby('team')['season'].shift(1)

# Rest days only within the same season (avoid giant off-season gaps)
rest_days = (df['date'] - df['__prev_date']).dt.days
df['rest_days'] = np.where(df['season'].eq(df['__prev_season']), rest_days, np.nan)

# Helpful indicators (set to <NA> when rest_days is NaN)
df['short_rest'] = (df['rest_days'] <= 6).astype('Int64')
df['bye_week']   = (df['rest_days'] >= 13).astype('Int64')
df.loc[df['rest_days'].isna(), ['short_rest', 'bye_week']] = pd.NA

# Drop helper columns
df.drop(columns=['__prev_date','__prev_season'], inplace=True)


Print to the new intermediate dataset

In [14]:
# save
df.to_csv("../intermediate/schedules_cleaned.csv", index=False)
print(df.head())

             game_id  season  week       date team opponent  is_home  \
0     2014_01_SD_ARI    2014     1 2014-09-08  ARI       SD        1   
32   2014_02_ARI_NYG    2014     2 2014-09-14  ARI      NYG        0   
64    2014_03_SF_ARI    2014     3 2014-09-21  ARI       SF        1   
122  2014_05_ARI_DEN    2014     5 2014-10-05  ARI      DEN        0   
152  2014_06_WAS_ARI    2014     6 2014-10-12  ARI      WAS        1   

     team_score  opp_score  team_win  ...  opp_fgm_prior1 opp_xpa_prior1  \
0            18         17         1  ...             NaN            NaN   
32           25         14         1  ...             0.0            2.0   
64           23         14         1  ...             2.0            2.0   
122          20         41         0  ...             1.0            1.0   
152          30         20         1  ...             1.0            2.0   

    opp_xpm_prior1  opp_plays_offense_prior1  opp_score_prior1  \
0              NaN                       NaN

In [15]:
print(df.columns.tolist())

['game_id', 'season', 'week', 'date', 'team', 'opponent', 'is_home', 'team_score', 'opp_score', 'team_win', 'point_diff', 'home_team', 'away_team', 'home_score', 'away_score', 'Winner', 'spread_line', 'total_line', 'stadium', 'roof', 'surface', 'temp', 'wind', 'game_type', 'weekday', 'gametime', 'location', 'referee', 'team_coach', 'team_fga', 'team_fgm', 'team_first_down', 'team_fumbles_lost', 'team_moneyline', 'team_pass_att', 'team_pass_cmp', 'team_pass_int', 'team_pass_sacked', 'team_pass_td', 'team_pass_yds', 'team_penalties', 'team_penalties_yds', 'team_plays_offense', 'team_rush_att', 'team_rush_td', 'team_rush_yds', 'team_turnovers', 'team_xpa', 'team_xpm', 'opp_coach', 'opp_fga', 'opp_fgm', 'opp_first_down', 'opp_fumbles_lost', 'opp_moneyline', 'opp_pass_att', 'opp_pass_cmp', 'opp_pass_int', 'opp_pass_sacked', 'opp_pass_td', 'opp_pass_yds', 'opp_penalties', 'opp_penalties_yds', 'opp_plays_offense', 'opp_rush_att', 'opp_rush_td', 'opp_rush_yds', 'opp_turnovers', 'opp_xpa', 'opp